In [78]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pathlib import Path

# Code for getting usefull EnergyPlan `input`-data

Here I clean the KF- and EnergyDataService data.

## 1. KF-data

In [79]:
timeseries = ['Solceller','Landvind','Havvind']

for t in timeseries:

    # 0. read
    df = pd.read_excel('0_KF_data/KF25 Elsystem -  timedata (4).xlsx', sheet_name=t)

    # 1. clean
    col1 = df.iloc[0].astype(str)
    col2 = df.iloc[1].astype(str)
    df.columns = col1 + "_" + col2
    df = df.iloc[2:].reset_index(drop=True)

    # 2. sum for same year
    years = df.columns.str.extract(r'(\d{4})$', expand=False)
    df2 = df.loc[:, years.notna()].copy()
    years2 = years[years.notna()]
    df_yearsum = df2.groupby(years2, axis=1).sum()

    # 3. keep only 2035
    df_2035 = df_yearsum[['2035']]

    # 4. add first 24 to end (energyplan-format)
    df_2035 = pd.concat([df_2035, df_2035.iloc[0:24]], ignore_index=True)

    # 4. save as .txt
    df_2035.to_csv(
    r'..\ZipEnergyPLAN163\energyPlan Data\Distributions\KF25_2035_' + t + '.txt',
    sep='\t',
    index=False,
    header=False
    )

C:\Users\LinusLindquist\AppData\Local\Temp\ipykernel_17496\270556965.py:18: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_yearsum = df2.groupby(years2, axis=1).sum()
C:\Users\LinusLindquist\AppData\Local\Temp\ipykernel_17496\270556965.py:18: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_yearsum = df2.groupby(years2, axis=1).sum()
C:\Users\LinusLindquist\AppData\Local\Temp\ipykernel_17496\270556965.py:18: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_yearsum = df2.groupby(years2, axis=1).sum()


## 2. EnergyDataService

From EDS I download 'Electricity demand' and 'External Electricity Market price distribution' for 2024 initially.

### 2.1 Elspot price

In [80]:
# 0. get using API
BASE = "https://api.energidataservice.dk/dataset/Elspotprices"

params = {
    "start":    "2024-01-01T00:00",             
    "end":      "2025-01-01T00:00",
    "timezone": "UTC",                       
    "filter":   '{"PriceArea":["DK1","DK2"]}',              # Q til FL
    "columns":  "HourUTC,HourDK,PriceArea,SpotPriceEUR",
    "sort":     "HourUTC asc",
    "limit":    0                                    
}

r = requests.get(BASE, params=params, timeout=60)
r.raise_for_status()
df = pd.DataFrame(r.json().get("records", []))
df["HourUTC"] = pd.to_datetime(df["HourUTC"])

# 1. conpute average
df_avg = (
    df.groupby("HourUTC")["SpotPriceEUR"]
      .mean()
      .to_frame("SpotPriceEUR")   # keep as DataFrame
)

# 2. extract year of first row from the index
year = df_avg.index[0].year

# 3. save as .txt
df_avg.to_csv(
    fr'..\ZipEnergyPLAN163\energyPlan Data\Distributions\{year}_ENS_elspotprices.txt',
    sep='\t',
    index=False,
    header=False,
)

### 2.2 Demand distribution

In [81]:
# 0. get using API
BASE = "https://api.energidataservice.dk/dataset/ProductionConsumptionSettlement"

params = {
    "start":    "2024-01-01T00:00",             
    "end":      "2025-01-01T00:00",
    "timezone": "UTC",                       
    "columns":  "HourUTC,PriceArea,GrossConsumptionMWh",
    "sort":     "HourUTC asc",
    "limit":    0                                    
}

r = requests.get(BASE, params=params, timeout=60)
r.raise_for_status()
df = pd.DataFrame(r.json().get("records", []))
df["HourUTC"] = pd.to_datetime(df["HourUTC"])

# 1. sum for DK
df_dk = df.groupby("HourUTC")["GrossConsumptionMWh"].sum().to_frame("GrossConsumptionMWh")

# 2. extract year of first row from the index
year = df_dk.index[0].year

# 3. save as .txt
df_dk.to_csv(
    fr'..\ZipEnergyPLAN163\energyPlan Data\Distributions\{year}_ENS_ProductionConsumptionSettlement.txt',
    sep='\t',
    index=False,
    header=False,
)